## Load Model From huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModel

device = 'cuda'
checkpoint = "THUDM/chatglm-6b"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True)


## Insert LoRA to model

In [3]:
import loralib as lora
import lora_utils.insert_lora

In [4]:
lora_config = {
        'r': 32,
        'lora_alpha':32,
        'lora_dropout':0.1,
        'enable_lora':[True, True, True],
    }

In [5]:
model = lora_utils.insert_lora.get_lora_model(model, lora_config)

trainable_params:22020096 (0.35%), non_trainable_params:6255206400


## Dataset

In [9]:
import dataset.GLM as GLM_Data
from torch.utils.data import DataLoader

GLM_Data.device = device

In [10]:
pairs = [{'prompt':'Hello!', 'completion':'Hi! This is ChatGLM.'}]
pairs_encoded = GLM_Data.encode_pairs(pairs, tokenizer)
train_dataset = GLM_Data.GLMDataset(pairs_encoded)
train_dataloader = DataLoader(dataset=train_dataset, collate_fn = GLM_Data.collate_fn, shuffle=True, batch_size=1)

In [11]:
model.half().to(device)
batch = {k: v.to(device) for k, v in next(iter(train_dataloader)).items()}
outputs = model(**batch)

In [12]:
outputs.loss

tensor(3.6348, device='cuda:0', dtype=torch.float16, grad_fn=<ToCopyBackward0>)